In [1]:
from pyspark import SparkContext, SparkConf
import collections
import os
os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk1.8.0_231"
os.environ["SPARK_HOME"] = "C:\\Users\\yizho\\Downloads\\spark-2.4.4-bin-hadoop2.7\\spark-2.4.4-bin-hadoop2.7"
os.environ['HADOOP_HOME'] = "C:\\winutils"
conf = SparkConf().setMaster("local").setAppName("payment")
sc = SparkContext(conf=conf)
sc.parallelize([0, 2, 3, 4, 6], 5).glom().collect()
lines = sc.textFile("C:/Users/yizho/PycharmProjects/HW8/payment.csv")
noHeaderRDD = lines.zipWithIndex().filter(lambda row_index: row_index[1] > 0).keys()
noHeaderRDD.first()

'1,1,1,76,2.99 ,2005/5/25 11:30,2006/2/15 22:12'

In [2]:
def parseLine(line):
    fields = line.split(',')
    datetimedata = fields[5]
    datetimearray = datetimedata.split('/')
    monthyear = datetimearray[0]+'0'+datetimearray[1]
    adjclose = float(fields[4])
    return (monthyear, adjclose)


rdd = noHeaderRDD.map(parseLine)
rdd.first()
rdd.mapValues(lambda x: (x, 1)).first()
priceByMonthYear = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
averagesByMonthYear = priceByMonthYear.mapValues(lambda x: x[0] / x[1])
results = averagesByMonthYear.collect()
for result in results:
    print (result)

('200505', 4.169775280898756)
('200506', 4.166038062283568)
('200507', 4.22796751601904)
('200508', 4.232834534904548)
('200602', 2.8251648351648404)


In [5]:
def customid(line):
    fields = line.split(',')
    adjclose = float(fields[4])
    customid = fields[1]
    return (customid, adjclose)


rdd = noHeaderRDD.map(customid)
rdd.first()
customtotalpayment = rdd.reduceByKey(lambda x,y : x + y)
results = customtotalpayment.collect()
for i in range(3):
    print (results[i])

('1', 118.67999999999992)
('2', 128.72999999999993)
('3', 135.73999999999998)


In [6]:
def staffid(line):
    fields = line.split(',')
    adjclose = float(fields[4])
    staffid = fields[2]
    return (staffid, adjclose)


rdd = noHeaderRDD.map(staffid)
rdd.first()
stafftotalpayment = rdd.reduceByKey(lambda x,y : x + y)
results = stafftotalpayment.collect()
for result in results:
    print (result)

('1', 33489.470000005174)
('2', 33927.04000000482)


In [7]:
def parseLine2(line):
    fields = line.split(',')
    datetimedata = fields[5]
    datetimearray = datetimedata.split('/')
    monthyear = datetimearray[0] + '0' + datetimearray[1]
    adjclose = float(fields[4])
    customid = fields[1]
    return ((monthyear, customid), adjclose)

rdd = noHeaderRDD.map(parseLine2)
rdd.first()
rdd.mapValues(lambda x: (x, 1)).first()
priceByMonthYear = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
averagesByMonthYear = priceByMonthYear.mapValues(lambda x: x[0] / x[1])
results = averagesByMonthYear.collect()
for i in range(8):
    print(results[i])

(('200505', '1'), 1.9900000000000002)
(('200506', '1'), 4.561428571428571)
(('200507', '1'), 4.240000000000001)
(('200508', '1'), 2.899090909090909)
(('200505', '2'), 4.99)
(('200506', '2'), 2.99)
(('200507', '2'), 5.418571428571428)
(('200508', '2'), 4.080909090909092)
